In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import urllib.request
from pathlib import Path
pd.set_option('display.max_colwidth', None)

In [2]:
recipe_url_db_foodnetwork_file = Path('.') / 'recipe_url_db_foodnetwork.parquet'
recipe_url_db_foodnetwork = pd.read_parquet(recipe_url_db_foodnetwork_file)

In [16]:
def recipe_master_db_foodnetwork(database, img_download_path=Path('images')):
    
    from foodnetwork import (get_ingredients_foodnetwork, get_instructions_foodnetwork, 
                             get_images_foodnetwork, get_cooking_time_foodnetwork, 
                             get_difficulty_foodnetwork, get_tags_foodnetwork)
    
    
    driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
    master_data = {'name': [], 'ingredients': [], 'instructions': [], 'images': [], 'cooking_time': [], 'difficulty': [], 'tags': [], 'url': []}
    
    for ix, r in tqdm(database.iterrows(), total=len(database)):
        name = r.loc['recipe_name']
        url = r.loc['recipe_url']
        driver.get(url)
        page_content = BeautifulSoup(driver.page_source)
        h1 = page_content.find('body').find('h1')
        if h1 and re.search("500|404|403", h1.text) is not None:
            continue
        
        # Obtain master data
        master_data['name'].append(name)
        master_data['url'].append(url)
        master_data['ingredients'].append(get_ingredients_foodnetwork(page_content))
        master_data['instructions'].append(get_instructions_foodnetwork(page_content))
        master_data['images'].append(get_images_foodnetwork(page_content, img_download_path, name.lower()))
        master_data['cooking_time'].append(get_cooking_time_foodnetwork(page_content))
        master_data['difficulty'].append(get_difficulty_foodnetwork(page_content))
        master_data['tags'].append(get_tags_foodnetwork(page_content))

    master_data_df = pd.DataFrame(master_data)
    
    return master_data_df

master_data = recipe_master_db_foodnetwork(recipe_url_db_foodnetwork, Path('images'))

Automatic pdb calling has been turned ON


In [19]:
recipe_url_db_foodnetwork.shape

(456, 2)

In [21]:
master_data[0].head()

,name,ingredients,instructions,images,cooking_time,difficulty,tags,url
0,Millionaire's Shortbread,"[125g butter or margarine, softened, plus extr...","[1. Preheat the oven to 180°C, gas mark 4 and ...",[https://d2v9mhsiek5lbq.cloudfront.net/eyJidWN...,25 mins,Easy,"[margarine, sugar, flour, chocolate, milk, syr...",https://foodnetwork.co.uk/recipes/millionaires...
1,Lamb and Pea Pie,"[1 tbsp sunflower oil by Sainsbury’s, 1 onion,...","[Preheat the oven to 220°C, fan 200°C, gas 7. ...",[https://d2v9mhsiek5lbq.cloudfront.net/eyJidWN...,30 mins,Easy,"[oil, onion, garlic, lamb, soy sauce, mustard,...",https://foodnetwork.co.uk/recipes/lamb-and-pea...
2,Beef Tacos,"[3/4 cup peanut oil, 12 (6-inch) yellow corn t...",[Make a taco mold by rolling out aluminum foil...,[https://d2v9mhsiek5lbq.cloudfront.net/eyJidWN...,30 mins,Medium,"[peanut, tortilla, onion, sirloin, garlic, bee...",https://foodnetwork.co.uk/recipes/all-american...
3,Cheese and Mushroom Omelette,"[4 The Co-operative British free range eggs, 6...",[1. Beat the eggs well and add in some black p...,[https://d2v9mhsiek5lbq.cloudfront.net/eyJidWN...,10 mins,Easy,"[egg, cheddar, mushroom, pepper, Main Course, ...",https://foodnetwork.co.uk/recipes/cheese-and-m...
4,Lemongrass-Coconut Prawn and Noodles Parchment...,"[4 ounces rice vermicelli noodles, 4 cups baby...",[1. Position 2 racks in the upper and lower th...,[https://d2v9mhsiek5lbq.cloudfront.net/eyJidWN...,15 mins,Easy,"[vermicelli, spinach, carrot, coconut milk, cu...",https://foodnetwork.co.uk/recipes/lemongrass-c...
